<a href="https://colab.research.google.com/github/shanemsaito/musicyearidentification/blob/main/Trained_Music_Decade_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class Model(nn.Module):
  def __init__(self, in_features = 14, h1 = 8, h2 = 9, out_features = 7):
    super().__init__()
    self.fc1 = nn.Linear(in_features, h1)
    self.fc2 = nn.Linear(h1, h2)
    self.out = nn.Linear(h2, out_features)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.out(x)

    return x

In [ ]:
torch.manual_seed(41)
model = Model()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
data_frame = pd.read_csv('sample_data/music_decade_training_set_revised.csv')


In [ ]:
data_frame

,Unnamed: 0,playlist,loudness,timbre_value_1,timbre_value_2,timbre_value_3,timbre_value_4,timbre_value_5,timbre_value_6,timbre_value_7,timbre_value_8,timbre_value_9,timbre_value_10,timbre_value_11,timbre_value_12
0,0,0.0,-9.129,47.563816,61.243679,63.988015,20.331497,30.251624,-6.723331,3.959830,0.800494,7.280230,4.154988,-15.875887,-5.066963
1,1,0.0,-13.710,40.642055,-30.339963,1.281939,-0.206732,34.191011,-13.647347,-10.115816,-1.719708,11.622797,5.384392,-9.583574,3.276826
2,2,0.0,-7.456,47.064680,53.267249,5.985444,-23.452010,22.917112,-8.966594,-33.004197,-1.505742,-3.059006,0.303590,-10.270905,-1.443154
3,3,0.0,-4.729,48.066179,-11.699396,0.816827,-6.134247,30.368631,4.248505,-13.185054,4.222040,-7.030671,-5.519231,-14.499282,1.926971
4,4,0.0,-13.507,41.929908,-51.424920,-2.318594,-22.295684,17.064933,-22.718628,-10.666163,3.398421,-0.655735,5.988196,-7.544250,2.219772
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
678,695,6.0,-4.721,50.147652,10.927541,27.532173,-5.541458,45.003893,-23.434137,3.299378,0.223994,-2.423903,-0.789259,-8.643053,4.048974
679,696,6.0,-5.802,46.947397,-5.943652,-10.897856,-15.452240,35.318613,-29.632662,2.191802,-3.095579,-19.009253,-5.799496,-7.265856,-7.870397
680,697,6.0,-6.694,49.219826,23.536434,5.726556,-4.046505,34.969340,-22.203700,-7.128264,-4.411882,-7.038135,0.945401,-8.466934,2.463005
681,698,6.0,-4.808,51.472445,49.137614,-6.263000,-7.452964,8.020083,-13.777622,-21.023736,-0.083716,2.052594,5.552538,-10.702997,-4.793122


In [ ]:
x = data_frame.drop('playlist', axis = 1)
y = data_frame['playlist']

In [ ]:
X = x.values
Y = y.values

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 41)

In [ ]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

In [ ]:
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)


In [ ]:
epoch = 100
losses = []

for i in range(epoch):
  y_pred = model.forward(X_train)

  loss = criterion(y_pred, y_train)

  losses.append(loss.detach().numpy())

  if i % 10 == 0:
    print(f'Epoch: {i} and loss : {loss}')

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()



Epoch: 0 and loss : 18.77198028564453
Epoch: 10 and loss : 1.9013229608535767
Epoch: 20 and loss : 1.404064416885376
Epoch: 30 and loss : 1.1464192867279053
Epoch: 40 and loss : 0.9642030000686646
Epoch: 50 and loss : 0.7651549577713013
Epoch: 60 and loss : 0.5373904705047607
Epoch: 70 and loss : 0.3607480525970459
Epoch: 80 and loss : 0.26561325788497925
Epoch: 90 and loss : 0.19955606758594513


In [ ]:
with torch.no_grad():
  y_eval = model.forward(X_test)
  loss = criterion(y_eval, y_test)

In [ ]:
correct = 0
with torch.no_grad():
    for i, data in enumerate(X_test):
      y_val = model.forward(data)

      print(f'{i+1}.) {str(y_val)}')

      #Correct or not
      if y_val.argmax().item() == y_test[i]:
        correct += 1

print(f'We got {correct} correct!')

1.) tensor([ -2.9225,  -3.1275,   7.1932,   7.1776,   2.9202, -14.6674, -26.5548])
2.) tensor([ -4.4994, -14.9300, -15.8842,  -4.7386,  10.4910,  16.8779,  23.5265])
3.) tensor([ -2.6872,  10.3021,   8.6466,   1.8852,  -3.6447, -29.5016, -46.4225])
4.) tensor([ -0.5852, -10.5693,  -5.4513,   3.4489,   9.3590,  11.1612,   5.1412])
5.) tensor([ -3.9349,   6.6123,   9.9817,   4.3367,  -0.4290, -21.9482, -37.4923])
6.) tensor([-1.2612, -6.7985,  0.4516,  5.3574,  6.3883, -0.5262, -9.8719])
7.) tensor([ -0.8396,  -4.6030,   3.8137,   6.9152,   3.9747,  -8.3855, -19.7884])
8.) tensor([  0.3408,  -6.9034,   2.4752,   8.0104,   6.0954,  -1.7346, -14.0559])
9.) tensor([-2.6964, -9.7012, -6.3814,  0.7826,  9.6514, 12.1712,  8.8011])
10.) tensor([ -4.9048, -11.4601, -11.0499,  -3.4986,   9.6700,  14.3717,  18.0211])
11.) tensor([  8.3731,  14.6100,   0.6036,  -9.6004, -13.3161, -45.3857, -67.8520])
12.) tensor([ -4.9813, -10.7457,  -9.2600,  -2.6472,   9.8746,  14.4384,  16.5949])
13.) tensor([ -